**Processamento de Linguagem Natural (PLN)**
Natural Language processing (NLP)

**Análise de sentimento -Twitter**

idetificar se o tweet é racista/sexista ou não

In [1]:
import pandas as pd
import seaborn as sns
import re
import matplotlib.pyplot as plt
%matplotlib inline

import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer


import re

from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix, classification_report

#fazer download dos pacotes extras
nltk.download()


In [ ]:
# ler dados
test_csv = 'twitter/test.csv'
train_csv = 'twitter/train.csv'



: 

In [ ]:
test = pd.read_csv(test_csv)
train = pd.read_csv(train_csv)

: 

In [ ]:
train.shape,test.shape

: 

In [ ]:
train.head()

: 

**Pré- Processamento de Dados**
                                           
 tirar porntuação e caracteres especiais









In [ ]:
def limpa_texto(data,col):
    """
    criar função para remover pontuações e caracteres especiais,recebendo um dataframe(data) e uma coluna(col)
    """
    data[col]=data[col].str.lower()
    data[col] = data[col].apply(lambda x: re.sub(r'[^a-zA-Z0-9\s]', '', x))


: 

In [ ]:
limpa_texto(train,'tweet')

: 

In [ ]:
limpa_texto(test,'tweet')

: 

**Remover palavras comuns**

In [ ]:
from nltk.corpus import stopwords

def remove_palavras_comuns(data, col):
    palavras_comuns = set(stopwords.words('english'))

    # Itera sobre cada linha do DataFrame
    for idx, _ in data.iterrows():
        # Filtra palavras comuns
        palavras_sem_comuns = [palavra for palavra in data.loc[idx, col].split() if palavra not in palavras_comuns]
        # Atualiza a coluna 'col' com a lista de palavras sem as comuns
        data.at[idx, col] = palavras_sem_comuns



: 

In [ ]:
remove_palavras_comuns(train, 'tweet')

: 

In [ ]:
train['tweet']

: 

In [ ]:
train.head()

: 

In [ ]:
remove_palavras_comuns(test, 'tweet')

: 

In [ ]:
test.head()

: 

**Remover variações da mesma palavra**

In [ ]:
#identificar palavras diferentes e colocar no padrão
tira_variacao = WordNetLemmatizer()

: 

In [ ]:
#colocar o verbo no infinitivo
tira_variacao.lemmatize('went','v')

: 

In [ ]:
def remover_variacao_palavra(data, col):
  #identificar palavras diferentes e colocar no padrão
    tira_variacao = WordNetLemmatizer()
    for idx, _ in data.iterrows():
      data.at[idx, col]= [tira_variacao.lemmatize(palavra,'v')\
                          for palavra in  data[col][idx]]


: 

In [ ]:
remover_variacao_palavra(train, 'tweet')

: 

In [ ]:
remover_variacao_palavra(test, 'tweet')

: 

**Engenharia de Recurso(Feature Engineering)**

In [ ]:
train['tweet']

: 

In [ ]:
#quantidade de palavras por tweet
train['palavras']=train['tweet'].apply(len)

: 

In [ ]:
test['palavras']=test['tweet'].apply(len)

: 

**Análise Exploratória dos Dados(AED)**                                                   

Distribuição de cada 'label

In [ ]:
# Criando a coluna 'palavras' que conta o número de palavras em cada tweet
train['palavras'] = train['tweet'].apply(lambda x: len(x))

# Plotando o histograma dividido por 'label'
train.hist(column='palavras', by='label', bins=10, figsize=(12, 6), edgecolor='black')




: 

**Quantidade de cada 'label'**

In [ ]:
sns.countplot(x='label', data=train)
plt.show() #saída dados desequilibrados


: 

**Palavras frequêntes**                                        

Averiguar palavras  mais frequente classificadas como "racistas ou sexistas"

In [ ]:
#label 1 considerado racista ou sexista
lista_de_listas = train.loc[train['label']==1]['tweet'].values
lista_de_listas

: 

In [ ]:
#pegar os elementos dentro das listas
#aplicar a compreensão de lista
palavras = [palavra for lista in lista_de_listas for palavra in lista]


: 

In [ ]:
#saber qualtas palavras tem alista de palavras
len(palavras)

: 

In [ ]:
#extrair frequência de palavras repetidas
import collections

: 

In [ ]:
def gera_fre_palavras(lista):
  return collections.Counter(lista)

: 

In [ ]:
frequencia_palavras=gera_fre_palavras(palavras)

: 

In [ ]:
#quantidade de palavras repetidas
len(frequencia_palavras)

: 

In [ ]:
lista_de_palavras = []
lista_de_frequencias = []
for palavra, frequencia in frequencia_palavras.items():
    lista_de_palavras.append(palavra)
    lista_de_frequencias.append(frequencia)


: 

In [ ]:
data_frequencia = pd.DataFrame({'palavras': lista_de_palavras, 'freq': lista_de_frequencias})


: 

In [ ]:
print(data_frequencia)

: 

In [ ]:
# Ordenar DataFrame por frequência em ordem decrescente
data_frequencia.sort_values('freq', ascending=False, inplace=True)



: 

In [ ]:
data_frequencia

: 

**Nuvem de palavras**


Palavras mais frequentes apareçam  maiores
e palavras menos frequenetes que apareçam menores

In [ ]:
from wordcloud import WordCloud

: 

In [ ]:
dict(data_frequencia[:10].values)

: 

In [ ]:
nuvem = WordCloud(background_color='white')

: 

In [ ]:
#palavras que representam sexistas e racistas
plt.figure(figsize=(10,8),dpi=100)
nuvem.generate_from_frequencies(dict(data_frequencia[:50].values))
plt.imshow(nuvem)
plt.axis('off')
plt.show()


: 

**Retornar o texto para o formato natural**




In [ ]:
train['tweet'] = train['tweet'].apply(lambda x: ' '.join(x))

: 

In [ ]:
test['tweet'] = test['tweet'].apply(lambda x: ' '.join(x))

: 

In [ ]:
test['tweet']

: 

**Solucionar problemas de desequilibrio dos dados**

o objetivo não é apenas ter uma boa acuracia em termos de acertar ou não simplesmentes colocando tudo zero.

In [ ]:
#contar valores por label
train['label'].value_counts()

: 

In [ ]:
#exemplo de otíma acuracidade, porém não detectar nenhum comentario ofensivo
29720/(29720+ 2242)

: 

In [ ]:
#avaliar o F1score
from sklearn.utils import resample

: 

In [ ]:
train_minoria =train.loc[train['label']==1]

: 

In [ ]:
train_maioria = train.loc[train['label']==0]

: 

In [ ]:
def equilibra_dados(train,pra_mais=True):
  train_minoria =train.loc[train['label']==1]
  train_maioria = train.loc[train['label']==0]
  if pra_mais:
    menor_pro_maior=resample(train_minoria,replace=True,
                         n_samples = len(train_maioria),
                         random_state = 123)
    #concatenando
    train_equilibrado = pd.concat([menor_pro_maior,train_maioria])
  else:
      maior_pro_maior = resample(train_maioria,replace=True,
                         n_samples=len(train_minoria),
                         random_state =123)
      #concatenando
      train_equilibrado = pd.concat([maior_pro_maior,train_minoria])
  return train_equilibrado


: 

In [ ]:
train_equilibrado = equilibra_dados(train,pra_mais=True)

: 

In [ ]:
len(train_equilibrado)

: 

In [ ]:
train_equilibrado['label'].value_counts()

: 

**Treinar modelo**

In [ ]:
from sklearn.model_selection import train_test_split

: 

In [ ]:
x = train_equilibrado['tweet']
y= train_equilibrado['label']

: 

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size =0.33,random_state=42)

: 

In [ ]:
x_train.shape, y_train.shape

: 

In [ ]:
x_test.shape, y_test.shape

: 

In [ ]:
#criando pipeline

sgd = Pipeline([('vect',CountVectorizer()),
                ('tfid',TfidfTransformer()),
                ('modelo',SGDClassifier())])

: 

In [ ]:
sgd.fit(x_train, y_train)

: 

In [ ]:
sgd_pred = sgd.predict(x_test)

: 

In [ ]:
sgd_pred

: 

**Avaliar a perfomance do modelo**

In [ ]:
from sklearn.metrics import f1_score
sgd_score = f1_score(y_test,sgd_pred)


: 

In [ ]:
sgd_score

: 

In [ ]:
y_pred = sgd.predict(test['tweet'])

: 

In [ ]:
y_pred

: 

**Submissão**

In [ ]:
submissao = pd.DataFrame({'id': test['id'],'labe':y_pred})

: 

In [ ]:
submissao.head()

: 

In [ ]:
submissao.to_csv('pre_twitter.csv',index=False)

: 